# Eyesense

Ocular Disease Intelligent Recognition (ODIR) is a structured ophthalmic database of 5,000 patients with age, color fundus photographs from left and right eyes and doctors' diagnostic keywords from doctors.

This dataset is meant to represent ‘‘real-life’’ set of patient information collected by Shanggong Medical Technology Co., Ltd. from different hospitals/medical centers in China. In these institutions, fundus images are captured by various cameras in the market, such as Canon, Zeiss and Kowa, resulting into varied image resolutions.
Annotations were labeled by trained human readers with quality control management. They classify patient into eight labels including:

-Normal (N),
-Diabetes (D),
-Glaucoma (G),
-Cataract (C),
-Age related Macular Degeneration (A),
-Hypertension (H),
-Pathological Myopia (M),
-Other diseases/abnormalities (O)

In [1]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import pathlib
from tensorflow.keras import layers
import numpy as np
import cv2
from PIL import Image, ImageEnhance
import random
from scipy.ndimage import gaussian_filter

2025-03-17 21:36:54.465433: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-17 21:36:54.691253: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-17 21:36:54.884740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742258215.055449   74225 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742258215.100913   74225 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742258215.503628   74225 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
df = pd.read_csv("raw_data/full_df.csv")

In [3]:
# We will drop rows with the classification "Others" because we believe that the model will have difficult learning it
df = df[df.labels != "['O']"]

In [4]:
# We have a disbalanced dataset
df["labels"].value_counts()

labels
['N']    2873
['D']    1608
['C']     293
['G']     284
['A']     266
['M']     232
['H']     128
Name: count, dtype: int64

In [55]:
def augment_fundus_image(image_path):
    """
    Load a fundus image, apply 8 different augmentations, and save the transformed versions
    with descriptive suffixes.
    
    Args:
        image_path (str): Path to the original fundus image
    """
    # Load the image
    img = Image.open(image_path)
    
    # Get the filename and directory
    img_dir, img_filename = os.path.split(image_path)
    filename, ext = os.path.splitext(img_filename)
    
    # 1. Rotation (±20 degrees)
    angle = random.uniform(-20, 20)
    rotated = img.rotate(angle, resample=Image.BICUBIC, expand=False)
    rotated_path = os.path.join(img_dir, f"{filename}_rot{angle:.1f}{ext}")
    rotated.save(rotated_path)
    
    # 2. Horizontal flip
    flipped_h = img.transpose(Image.FLIP_LEFT_RIGHT)
    flipped_h_path = os.path.join(img_dir, f"{filename}_fliph{ext}")
    flipped_h.save(flipped_h_path)
    
    # 3. Brightness adjustment (+15%)
    brightness_enhancer = ImageEnhance.Brightness(img)
    brightened = brightness_enhancer.enhance(1.15)
    brightened_path = os.path.join(img_dir, f"{filename}_bright+15{ext}")
    brightened.save(brightened_path)
    
    # 4. Brightness adjustment (-15%)
    darkened = brightness_enhancer.enhance(0.85)
    darkened_path = os.path.join(img_dir, f"{filename}_bright-15{ext}")
    darkened.save(darkened_path)
    
    # 5. Contrast adjustment (+15%)
    contrast_enhancer = ImageEnhance.Contrast(img)
    contrast_increased = contrast_enhancer.enhance(1.15)
    contrast_increased_path = os.path.join(img_dir, f"{filename}_contr+15{ext}")
    contrast_increased.save(contrast_increased_path)
    
    # 6. Slight Gaussian blur
    # Convert to numpy array for OpenCV processing
    img_np = np.array(img)
    blurred = cv2.GaussianBlur(img_np, (5, 5), 0)
    blurred_img = Image.fromarray(blurred)
    blurred_path = os.path.join(img_dir, f"{filename}_blur{ext}")
    blurred_img.save(blurred_path)
    
    # 7. Small translation
    width, height = img.size
    x_shift = int(width * 0.05)  # 5% shift
    y_shift = int(height * 0.05)  # 5% shift
    
    # Create a new image with the same size and black background
    translated = Image.new('RGB', (width, height))
    # Paste the original image with an offset
    translated.paste(img, (x_shift, y_shift))
    translated_path = os.path.join(img_dir, f"{filename}_trans{ext}")
    translated.save(translated_path)
    
    # 8. Slight scaling (zoom in 10%)
    scale_factor = 1.1
    
    # Calculate new dimensions
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    
    # Resize image
    scaled = img.resize((new_width, new_height), Image.BICUBIC)
    
    # Crop back to original size (from center)
    left = (new_width - width) // 2
    top = (new_height - height) // 2
    right = left + width
    bottom = top + height
    scaled = scaled.crop((left, top, right, bottom))
    
    scaled_path = os.path.join(img_dir, f"{filename}_zoom10{ext}")
    scaled.save(scaled_path)
    
    print(f"Successfully created 8 augmented versions of {img_filename}")

In [36]:
path = "data_train" # folder created in pre_processing
dir_list = os.listdir(path)

In [37]:
dir_list.remove('normal') # the mais class
dir_list.remove('diabets') # the second class
dir_list.remove('others') # we won't use this class
dir_list

['myopia', 'cataract', 'hypertension', 'degeneration', 'glaucoma']

In [53]:
for i in dir_list:
    
    i_path = os.path.join(path, i)
    print(i_path)
    
    image_list = os.listdir(i_path)

    image_path = [os.path.join(i_path, x) for x in image_list]
    
    [augment_fundus_image(x) for x in image_path]
    
    print(f"{i} done")

data_train/myopia
Successfully created 8 augmented versions of 1552_right.jpg
Successfully created 8 augmented versions of 1599_left.jpg
Successfully created 8 augmented versions of 1602_right.jpg
Successfully created 8 augmented versions of 1647_right.jpg
Successfully created 8 augmented versions of 1619_right.jpg
Successfully created 8 augmented versions of 1558_left.jpg
Successfully created 8 augmented versions of 1624_left.jpg
Successfully created 8 augmented versions of 1642_right.jpg
Successfully created 8 augmented versions of 1662_left.jpg
Successfully created 8 augmented versions of 46_right.jpg
Successfully created 8 augmented versions of 1554_right.jpg
Successfully created 8 augmented versions of 1578_right.jpg
Successfully created 8 augmented versions of 390_right.jpg
Successfully created 8 augmented versions of 16_right.jpg
Successfully created 8 augmented versions of 1527_left.jpg
Successfully created 8 augmented versions of 1588_left.jpg
Successfully created 8 augmented 